# Домашнее задание №1: Распознавание речи (ASR)

## Часть 1: Подготовка данных и разработка токенизатора

**Цель:** Освоить этапы подготовки данных и токенизации текста для ASR.

**Задачи:**
1. Разработать собственный токенизатор (372(-3) = 369 токенов)
2. Подготовить данные из СОВА и СберГолос
3. Создать манифесты для NeMo

## Часть 2: Обучение и анализ модели ASR

**Цель:** Закрепить навыки построения и обучения нейронных сетей для ASR.

**Задачи:**
1. Обучить нейронную сеть в пакете NeMo
2. Применить собственный токенизатор
3. Настроить процесс обучения, зафиксировать метрики (loss, accuracy, WER)
4. Построить графики обучения
5. Провести тестирование модели
6. Сохранить модель в формате .nemo

## Установка зависимостей


In [ ]:
# Установка NeMo и необходимых библиотек
%pip install nemo_toolkit[all] -q
%pip install setuptools wheel -q
%pip install tensorboard -q
%pip install jiwer -q  # Для вычисления WER
%pip install tokenizers -q  # Для создания токенизатора
%pip install nvidia-ml-py -q  # Для замены устаревшего pynvml

print("✓ Зависимости установлены")


## Импорты и настройка seed

In [ ]:
import os
import json
import glob
import random
import re
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import librosa
import soundfile as sf
import torch

# NeMo импорты
try:
    import nemo
    import nemo.collections.asr as nemo_asr
    from nemo.collections.asr.models import ASRModel
    from nemo.core.config import hydra_runner
    from nemo.utils import logging
    print("✓ NeMo успешно импортирован")
except ImportError as e:
    print(f"⚠️  Ошибка импорта NeMo: {e}")
    print("   Убедитесь, что NeMo установлен: pip install nemo_toolkit[all]")

In [ ]:
# Настройка seed для воспроизводимости
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

os.environ['PL_GLOBAL_SEED'] = str(SEED)

print(f"✓ Seed установлен: {SEED}")
print(f"✓ PL_GLOBAL_SEED: {os.environ.get('PL_GLOBAL_SEED')}")

## Определение путей

## Часть 1: Разработка токенизатора

### Требования:
- Количество токенов: **372(-3) = 369 токенов**
- Стратегия: BPE (Byte Pair Encoding) или Unigram Language Model
- Токенизатор должен быть обучен на данных из СОВА и СберГолос


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram
from tokenizers.trainers import BpeTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import BertProcessing

# Целевое количество токенов
TARGET_VOCAB_SIZE = 369  # 372 - 3

print(f"Целевой размер словаря: {TARGET_VOCAB_SIZE} токенов")


In [ ]:
def normalize_text_for_asr(text: str) -> str:
    """Нормализация текста для ASR."""
    # Приводим к нижнему регистру
    text = text.lower().strip()
    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    # Убираем специальные символы, оставляем только буквы, цифры и пробелы
    text = re.sub(r'[^а-яёa-z0-9\s]', '', text)
    return text.strip()

def load_texts_from_manifests(manifest_paths: List[str]) -> List[str]:
    """Загружает тексты из манифестов NeMo."""
    texts = []
    for manifest_path in manifest_paths:
        if not os.path.exists(manifest_path):
            print(f"⚠️  Манифест не найден: {manifest_path}")
            continue
        
        with open(manifest_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    text = data.get('text', '')
                    if text:
                        texts.append(normalize_text_for_asr(text))
                except json.JSONDecodeError:
                    continue
    return texts

print("✓ Функции для загрузки текстов готовы")


In [ ]:
def train_bpe_tokenizer(texts: List[str], vocab_size: int = TARGET_VOCAB_SIZE, output_path: Optional[str] = None) -> Tokenizer:
    """Обучает BPE токенизатор на текстах."""
    # Создаем токенизатор
    tokenizer = Tokenizer(BPE(unk_token="<unk>"))
    tokenizer.pre_tokenizer = Whitespace()
    
    # Создаем тренер
    trainer = BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=["<unk>", "<s>", "</s>", "<pad>"]
    )
    
    # Обучаем токенизатор
    print(f"Обучение BPE токенизатора на {len(texts)} текстах...")
    tokenizer.train_from_iterator(texts, trainer)
    
    # Сохраняем токенизатор
    if output_path:
        tokenizer.save(output_path)
        print(f"✓ Токенизатор сохранен: {output_path}")
    
    return tokenizer

print("✓ Функция обучения BPE токенизатора готова")


## Подготовка данных


In [ ]:
def load_audio_info(audio_path: str) -> Tuple[Optional[float], Optional[int], Optional[np.ndarray]]:
    """Загружает информацию об аудиофайле."""
    try:
        y, sr = librosa.load(audio_path, sr=None)
        duration = len(y) / sr
        return duration, sr, y
    except Exception as e:
        print(f"Ошибка при загрузке {audio_path}: {e}")
        return None, None, None

def prepare_manifest_from_dataset(
    dataset_path: str,
    output_manifest: str,
    dataset_name: str = "dataset",
    max_files: Optional[int] = None,
    audio_extensions: List[str] = ['.wav', '.mp3', '.flac', '.ogg']
) -> int:
    """Подготавливает манифест NeMo из датасета."""
    print(f"\nОбработка датасета: {dataset_name}")
    print(f"Путь: {dataset_path}")
    
    if not os.path.exists(dataset_path):
        print(f"⚠️  Путь не существует: {dataset_path}")
        return 0
    
    # Собираем все аудиофайлы
    audio_files = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in audio_extensions):
                audio_files.append(os.path.join(root, file))
    
    if max_files:
        audio_files = audio_files[:max_files]
    
    print(f"Найдено {len(audio_files)} аудиофайлов")
    
    # Обрабатываем файлы
    manifest_data = []
    processed = 0
    skipped = 0
    
    for audio_file in tqdm(audio_files, desc=f"Обработка {dataset_name}"):
        # Пытаемся найти соответствующий текстовый файл
        base_name = os.path.splitext(audio_file)[0]
        text_file = base_name + '.txt'
        
        # Если текстовый файл не найден, пробуем другие варианты
        if not os.path.exists(text_file):
            parent_dir = os.path.dirname(audio_file)
            text_file = os.path.join(parent_dir, os.path.basename(base_name) + '.txt')
        
        # Загружаем аудио
        duration, sr, audio = load_audio_info(audio_file)
        if duration is None or duration < MIN_DURATION or duration > MAX_DURATION:
            skipped += 1
            continue
        
        # Загружаем текст
        text = ""
        if os.path.exists(text_file):
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    text = f.read().strip()
            except:
                pass
        
        # Если текст не найден, пропускаем файл
        if not text:
            skipped += 1
            continue
        
        # Нормализуем текст
        text = normalize_text_for_asr(text)
        
        # Добавляем в манифест
        manifest_data.append({
            "audio_filepath": os.path.abspath(audio_file),
            "duration": duration,
            "text": text
        })
        processed += 1
    
    # Сохраняем манифест
    with open(output_manifest, 'w', encoding='utf-8') as f:
        for item in manifest_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"✓ Обработано: {processed}, пропущено: {skipped}")
    print(f"✓ Манифест сохранен: {output_manifest}")
    
    return processed

print("✓ Функции подготовки данных готовы")


In [ ]:
# OUTPUT_DIR делаем абсолютным, чтобы избежать проблем с дублированием путей
OUTPUT_DIR_REL = "./asr_data"
OUTPUT_DIR = os.path.abspath(os.path.normpath(OUTPUT_DIR_REL))

# Директории для данных
MANIFEST_DIR = os.path.join(OUTPUT_DIR, "manifests")
TOKENIZER_DIR = os.path.join(OUTPUT_DIR, "tokenizer")
MODEL_DIR = os.path.join(OUTPUT_DIR, "models")
TEST_DIR = os.path.join(OUTPUT_DIR, "test")
EXAMPLES_DIR = os.path.join(OUTPUT_DIR, "examples")

# Создаем директории
for dir_path in [OUTPUT_DIR, MANIFEST_DIR, TOKENIZER_DIR, MODEL_DIR, TEST_DIR, EXAMPLES_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print(f"OUTPUT_DIR: {OUTPUT_DIR}")
print(f"Текущая рабочая директория: {os.getcwd()}")

# Параметры аудио
SAMPLE_RATE = 16000  # Стандартная частота дискретизации для ASR
MIN_DURATION = 0.5
MAX_DURATION = 30.0

# ============================================================================
# ПОИСК ГОТОВЫХ МАНИФЕСТОВ (приоритет - использовать готовые)
# ============================================================================
print("\n" + "="*60)
print("ПОИСК ГОТОВЫХ МАНИФЕСТОВ")
print("="*60)

# Возможные расположения готовых манифестов
POSSIBLE_MANIFEST_LOCATIONS = [
    # Готовые train/val манифесты
    {
        'train': '/tf/sishmuratov/manifests/train.json',
        'val': '/tf/sishmuratov/manifests/val.json',
        'name': 'sishmuratov/manifests'
    },
    # Отдельные манифесты СОВА и СберГолос
    {
        'sova': '/tf/nspeganov/ASR/sova_manifest1.json',
        'golos': '/tf/nspeganov/ASR/golos_manifest.json',
        'name': 'nspeganov/ASR'
    },
    {
        'sova': '/tf/minazarko/asr/sem3ft/sova_manifest1.json',
        'golos': '/tf/minazarko/asr/golos_manifest.json',
        'name': 'minazarko/asr'
    }
]

# Ищем готовые train/val манифесты
train_manifest = None
val_manifest = None

for location in POSSIBLE_MANIFEST_LOCATIONS:
    if 'train' in location and 'val' in location:
        if os.path.exists(location['train']) and os.path.exists(location['val']):
            train_manifest = location['train']
            val_manifest = location['val']
            print(f"✓ Найдены готовые train/val манифесты в: {location['name']}")
            print(f"  Train: {train_manifest}")
            print(f"  Val: {val_manifest}")
            break

# Если не нашли готовые train/val, ищем отдельные манифесты СОВА и СберГолос
sova_manifest_found = None
golos_manifest_found = None

if not train_manifest or not val_manifest:
    print("\nИщем отдельные манифесты СОВА и СберГолос...")
    for location in POSSIBLE_MANIFEST_LOCATIONS:
        if 'sova' in location and 'golos' in location:
            if os.path.exists(location['sova']) and os.path.exists(location['golos']):
                sova_manifest_found = location['sova']
                golos_manifest_found = location['golos']
                print(f"✓ Найдены манифесты в: {location['name']}")
                print(f"  СОВА: {sova_manifest_found}")
                print(f"  СберГолос: {golos_manifest_found}")
                break

# Пути к исходным датасетам (используются только если манифесты не найдены)
# Локальные пути (приоритет)
LOCAL_SOVA_PATH = "/tf/nspeganov/sova_data"
LOCAL_SBER_PATH = "/tf/nspeganov/sber_golos_data"

# Share пути (fallback)
SOVA_PATH = "/share/audio_data/sova/ytub/raid/nanosemantics/nextcloud/sova_done"
SBER_GOLOS_PATH = "/share/audio_data/sber-golos/tar/train"

# Определяем, какие пути использовать (если понадобятся)
if os.path.exists(LOCAL_SOVA_PATH):
    SOVA_PATH = LOCAL_SOVA_PATH
    print(f"\n✓ Локальный путь СОВА найден: {SOVA_PATH}")
elif os.path.exists(SOVA_PATH):
    print(f"\n⚠️  Локальный путь СОВА не найден, используем share: {SOVA_PATH}")
else:
    print(f"\n⚠️  Путь СОВА не найден: {SOVA_PATH}")

if os.path.exists(LOCAL_SBER_PATH):
    SBER_GOLOS_PATH = LOCAL_SBER_PATH
    print(f"✓ Локальный путь СберГолос найден: {SBER_GOLOS_PATH}")
elif os.path.exists(SBER_GOLOS_PATH):
    print(f"⚠️  Локальный путь СберГолос не найден, используем share: {SBER_GOLOS_PATH}")
else:
    print(f"⚠️  Путь СберГолос не найден: {SBER_GOLOS_PATH}")

print("\n" + "="*60)


In [ ]:
# ============================================================================
# ПОДГОТОВКА МАНИФЕСТОВ
# ============================================================================

# Если уже нашли готовые train/val манифесты, используем их
if train_manifest and val_manifest and os.path.exists(train_manifest) and os.path.exists(val_manifest):
    print("\n✓ Используем готовые train/val манифесты, пропускаем подготовку")
    # Подсчитываем количество записей
    with open(train_manifest, 'r', encoding='utf-8') as f:
        train_count = sum(1 for line in f if line.strip())
    with open(val_manifest, 'r', encoding='utf-8') as f:
        val_count = sum(1 for line in f if line.strip())
    print(f"  Train: {train_count} записей")
    print(f"  Val: {val_count} записей")
else:
    # Если нашли отдельные манифесты СОВА и СберГолос, объединяем их
    if sova_manifest_found and golos_manifest_found:
        print("\n✓ Используем найденные манифесты СОВА и СберГолос")
        sova_manifest = sova_manifest_found
        sber_manifest = golos_manifest_found
    else:
        # Создаем манифесты заново
        print("\nПодготовка манифестов из исходных данных...")
        sova_manifest = os.path.join(MANIFEST_DIR, "sova_manifest.json")
        sber_manifest = os.path.join(MANIFEST_DIR, "sber_golos_manifest.json")
        
        # Проверяем, существуют ли уже созданные манифесты
        if os.path.exists(sova_manifest) and os.path.exists(sber_manifest):
            print("✓ Манифесты уже существуют в OUTPUT_DIR, используем их")
        else:
            # Подготавливаем манифесты
            print("Создание манифестов из исходных данных...")
            
            # СОВА датасет
            if os.path.exists(SOVA_PATH):
                prepare_manifest_from_dataset(
                    SOVA_PATH,
                    sova_manifest,
                    dataset_name="СОВА",
                    max_files=10000  # Ограничиваем для ускорения
                )
            else:
                print(f"⚠️  СОВА датасет не найден: {SOVA_PATH}")
            
            # СберГолос датасет
            if os.path.exists(SBER_GOLOS_PATH):
                prepare_manifest_from_dataset(
                    SBER_GOLOS_PATH,
                    sber_manifest,
                    dataset_name="СберГолос",
                    max_files=10000  # Ограничиваем для ускорения
                )
            else:
                print(f"⚠️  СберГолос датасет не найден: {SBER_GOLOS_PATH}")
    
    # Объединяем манифесты и создаем train/val
    train_manifest = os.path.join(MANIFEST_DIR, "train_manifest.json")
    val_manifest = os.path.join(MANIFEST_DIR, "val_manifest.json")
    
    # Проверяем, нужно ли создавать train/val манифесты
    if os.path.exists(train_manifest) and os.path.exists(val_manifest):
        print("\n✓ Train/val манифесты уже существуют, используем их")
        with open(train_manifest, 'r', encoding='utf-8') as f:
            train_count = sum(1 for line in f if line.strip())
        with open(val_manifest, 'r', encoding='utf-8') as f:
            val_count = sum(1 for line in f if line.strip())
        print(f"  Train: {train_count} записей")
        print(f"  Val: {val_count} записей")
    elif 'sova_manifest' in locals() and os.path.exists(sova_manifest) and os.path.exists(sber_manifest):
        # Объединяем и разделяем на train/val
        print("\nОбъединение манифестов и создание train/val...")
        all_data = []
        
        for manifest_path in [sova_manifest, sber_manifest]:
            if os.path.exists(manifest_path):
                with open(manifest_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        try:
                            all_data.append(json.loads(line.strip()))
                        except:
                            continue
        
        if all_data:
            # Перемешиваем
            random.shuffle(all_data)
            
            # Разделяем на train/val (90/10)
            split_idx = int(len(all_data) * 0.9)
            train_data = all_data[:split_idx]
            val_data = all_data[split_idx:]
            
            # Сохраняем
            with open(train_manifest, 'w', encoding='utf-8') as f:
                for item in train_data:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
            
            with open(val_manifest, 'w', encoding='utf-8') as f:
                for item in val_data:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
            
            print(f"\n✓ Объединенные манифесты созданы:")
            print(f"  Train: {len(train_data)} записей")
            print(f"  Val: {len(val_data)} записей")
        else:
            print("⚠️  Не удалось загрузить данные из манифестов")
    else:
        print("⚠️  Не найдены манифесты для создания train/val")
        print("   Убедитесь, что манифесты существуют или исходные данные доступны")

# Финальная проверка
if not train_manifest or not os.path.exists(train_manifest):
    train_manifest = os.path.join(MANIFEST_DIR, "train_manifest.json")
if not val_manifest or not os.path.exists(val_manifest):
    val_manifest = os.path.join(MANIFEST_DIR, "val_manifest.json")

print(f"\n{'='*60}")
print("ИТОГОВЫЕ ПУТИ К МАНИФЕСТАМ")
print(f"{'='*60}")
print(f"Train манифест: {train_manifest}")
print(f"Val манифест: {val_manifest}")
if os.path.exists(train_manifest) and os.path.exists(val_manifest):
    print("✓ Оба манифеста существуют и готовы к использованию")
else:
    print("⚠️  Один или оба манифеста отсутствуют!")


In [ ]:
# Обучение токенизатора на объединенных данных
tokenizer_path = os.path.join(TOKENIZER_DIR, "custom_tokenizer.json")

if os.path.exists(tokenizer_path):
    print("✓ Токенизатор уже обучен, загружаем...")
    tokenizer = Tokenizer.from_file(tokenizer_path)
else:
    print("Обучение токенизатора...")
    
    # Загружаем тексты из манифестов
    texts = load_texts_from_manifests([train_manifest, val_manifest])
    print(f"Загружено {len(texts)} текстов для обучения токенизатора")
    
    # Обучаем BPE токенизатор
    tokenizer = train_bpe_tokenizer(
        texts,
        vocab_size=TARGET_VOCAB_SIZE,
        output_path=tokenizer_path
    )

# Проверяем размер словаря
vocab = tokenizer.get_vocab()
actual_vocab_size = len(vocab)
print(f"\nРазмер словаря токенизатора: {actual_vocab_size} токенов")
print(f"Целевой размер: {TARGET_VOCAB_SIZE} токенов")

if actual_vocab_size == TARGET_VOCAB_SIZE:
    print("✓ Размер словаря соответствует требованиям!")
else:
    print(f"⚠️  Размер словаря не соответствует требованиям (разница: {actual_vocab_size - TARGET_VOCAB_SIZE})")

# Тестируем токенизатор
test_text = "привет как дела распознавание речи это интересно"
encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded.ids)

print(f"\nТест токенизатора:")
print(f"  Исходный текст: {test_text}")
print(f"  Токены: {encoded.tokens}")
print(f"  Количество токенов: {len(encoded.ids)}")
print(f"  Декодированный текст: {decoded}")

## Часть 2: Обучение модели NeMo ASR

## Визуализация метрик обучения


In [ ]:
# Визуализация метрик из TensorBoard (аналогично DZ2)
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

def plot_asr_training_metrics(log_dir):
    """Визуализирует метрики обучения ASR модели из TensorBoard."""
    print(f"Поиск логов TensorBoard...")
    print(f"Основная директория: {log_dir}")
    
    # Список возможных мест для поиска логов
    possible_log_dirs = [
        log_dir,  # Прямо в MODEL_DIR
        os.path.join(log_dir, "lightning_logs"),  # MODEL_DIR/lightning_logs
        os.path.join(OUTPUT_DIR, "lightning_logs"),  # OUTPUT_DIR/lightning_logs
        "/tf/lightning_logs",  # Корневая директория /tf
        os.path.join(os.path.dirname(OUTPUT_DIR), "lightning_logs"),  # Родительская директория
    ]
    
    version_dirs = []
    found_location = None
    
    # Ищем version_* директории во всех возможных местах
    for search_dir in possible_log_dirs:
        if os.path.exists(search_dir):
            print(f"  Проверяем: {search_dir}")
            # Ищем version_* напрямую
            found_versions = glob.glob(os.path.join(search_dir, "version_*"))
            if found_versions:
                version_dirs.extend(found_versions)
                if not found_location:
                    found_location = search_dir
                    print(f"  ✓ Найдены логи в: {search_dir}")
            # Также проверяем поддиректории
            for root, dirs, files in os.walk(search_dir):
                for d in dirs:
                    if d.startswith("version_"):
                        version_path = os.path.join(root, d)
                        if version_path not in version_dirs:
                            version_dirs.append(version_path)
                            if not found_location:
                                found_location = root
                                print(f"  ✓ Найдены логи в: {root}")
    
    if not version_dirs:
        print(f"\n⚠️  Логи TensorBoard не найдены ни в одном из проверенных мест:")
        for search_dir in possible_log_dirs:
            exists = "✓" if os.path.exists(search_dir) else "✗"
            print(f"  {exists} {search_dir}")
        print(f"\n💡 Возможные причины:")
        print(f"  1. Обучение еще не началось")
        print(f"  2. Обучение не завершило первую эпоху")
        print(f"  3. Логи находятся в другом месте")
        print(f"\n   Если обучение уже запущено, проверьте:")
        print(f"   - {log_dir}")
        print(f"   - {OUTPUT_DIR}")
        print(f"   - /tf/lightning_logs")
        return
    
    print(f"\n✓ Найдено {len(version_dirs)} директорий с логами")
    
    # Берем последнюю версию (по времени модификации)
    latest_version = max(version_dirs, key=os.path.getmtime)
    print(f"\nИспользуем логи из: {latest_version}")
    
    # Проверяем наличие event файлов
    event_files = glob.glob(os.path.join(latest_version, "events.out.tfevents.*"))
    if not event_files:
        print(f"⚠️  В директории {latest_version} не найдены event файлы TensorBoard")
        print(f"   Это нормально, если обучение только началось или еще не завершило первую эпоху.")
        return
    print(f"✓ Найдено {len(event_files)} event файлов")
    
    try:
        event_acc = EventAccumulator(latest_version)
        event_acc.Reload()
        
        scalar_tags = event_acc.Tags().get('scalars', [])
        print(f"Найдены теги: {scalar_tags}")
        
        if not scalar_tags:
            print("Нет данных для визуализации")
            return
        
        # Фильтруем служебные теги
        filtered_tags = [t for t in scalar_tags if 'hp_metric' not in t.lower() and 'hparams' not in t.lower()]
        
        # Определяем метрики для ASR
        loss_tags = [tag for tag in filtered_tags if 'loss' in tag.lower()]
        wer_tags = [tag for tag in filtered_tags if 'wer' in tag.lower()]
        accuracy_tags = [tag for tag in filtered_tags if 'accuracy' in tag.lower() or 'acc' in tag.lower()]
        lr_tags = [tag for tag in filtered_tags if 'lr' in tag.lower() or 'learning_rate' in tag.lower()]
        
        # Определяем количество графиков
        num_plots = sum([len(loss_tags) > 0, len(wer_tags) > 0, len(accuracy_tags) > 0, len(lr_tags) > 0])
        
        if num_plots == 0:
            print("Нет данных для визуализации")
            return
        
        # Создаем subplot
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.flatten()
        
        plot_idx = 0
        
        # График 1: Loss
        if loss_tags:
            ax = axes[plot_idx]
            for tag in loss_tags:
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            ax.set_xlabel('Step')
            ax.set_ylabel('Loss')
            ax.set_title('Training and Validation Loss')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
            plot_idx += 1
        
        # График 2: WER
        if wer_tags:
            ax = axes[plot_idx]
            for tag in wer_tags:
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            ax.set_xlabel('Step')
            ax.set_ylabel('WER')
            ax.set_title('Word Error Rate')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
            plot_idx += 1
        
        # График 3: Accuracy
        if accuracy_tags:
            ax = axes[plot_idx]
            for tag in accuracy_tags:
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            ax.set_xlabel('Step')
            ax.set_ylabel('Accuracy')
            ax.set_title('Accuracy')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
            plot_idx += 1
        
        # График 4: Learning Rate
        if lr_tags:
            ax = axes[plot_idx]
            for tag in lr_tags:
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            ax.set_xlabel('Step')
            ax.set_ylabel('LR')
            ax.set_title('Learning Rate')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
            plot_idx += 1
        
        # Скрываем неиспользованные subplot
        for i in range(plot_idx, len(axes)):
            axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'asr_training_metrics.png'), dpi=300)
        plt.show()
        
    except Exception as e:
        import traceback
        print(f"Ошибка при загрузке метрик: {e}")
        traceback.print_exc()

# Визуализируем метрики
print("="*60)
print("ВИЗУАЛИЗАЦИЯ МЕТРИК ОБУЧЕНИЯ")
print("="*60)
print("\n💡 Примечание: Если обучение еще не началось или не завершило первую эпоху,")
print("   логи TensorBoard могут отсутствовать. Запустите обучение модели сначала.")
print("\n" + "="*60 + "\n")

plot_asr_training_metrics(MODEL_DIR)


## Тестирование модели


In [ ]:
# Загрузка лучшей модели из чекпоинта
print("="*60)
print("ЗАГРУЗКА МОДЕЛИ ДЛЯ ТЕСТИРОВАНИЯ")
print("="*60)

checkpoints = glob.glob(os.path.join(MODEL_DIR, "checkpoints", "*.ckpt"))

if checkpoints:
    best_checkpoint = max(checkpoints, key=os.path.getmtime)
    print(f"\n✓ Найден чекпоинт: {best_checkpoint}")
    print(f"Загрузка модели из чекпоинта...")
    
    try:
        # Загружаем модель из чекпоинта
        asr_model = nemo_asr.models.EncDecCTCModel.load_from_checkpoint(best_checkpoint)
        asr_model.eval()
        print("✓ Модель загружена из чекпоинта")
    except Exception as e:
        print(f"⚠️  Ошибка при загрузке из чекпоинта: {e}")
        print("Используем текущую модель (если она загружена)")
else:
    print("\n⚠️  Чекпоинты не найдены")
    print("Используем текущую модель (если она загружена)")

# Проверяем, что модель загружена
if 'asr_model' not in globals() or asr_model is None:
    print("\n⚠️  Модель не загружена!")
    print("   Запустите ячейку 'Загрузка предобученной модели' или 'Обучение модели' сначала")
    raise RuntimeError("Модель не загружена. Загрузите модель перед тестированием.")

# Функция для распознавания речи
def transcribe_audio(audio_path: str, verbose: bool = False) -> str:
    """Распознает речь из аудиофайла."""
    if asr_model is None:
        if verbose:
            print(f"  ⚠️  Модель не загружена")
        return ""
    
    if not os.path.exists(audio_path):
        if verbose:
            print(f"  ⚠️  Файл не найден: {audio_path}")
        return ""
    
    try:
        if verbose:
            file_size = os.path.getsize(audio_path) / 1024  # KB
            print(f"  📁 Файл: {os.path.basename(audio_path)} ({file_size:.1f} KB)")
            print(f"  🔄 Начинаем распознавание...")
        
        transcription = asr_model.transcribe([audio_path])
        
        result = transcription[0] if transcription and len(transcription) > 0 else ""
        
        if verbose:
            if result:
                print(f"  ✓ Распознано: {result[:50]}...")
            else:
                print(f"  ⚠️  Пустой результат")
        
        return result
    except Exception as e:
        if verbose:
            print(f"  ✗ Ошибка при распознавании: {e}")
            import traceback
            traceback.print_exc()
        return ""

print("\n✓ Функция распознавания готова")
print(f"✓ Модель готова к использованию")


In [ ]:
# Тестирование на валидационной выборке
from jiwer import wer, cer
from IPython.display import Audio, display

def fix_audio_paths_in_manifest(manifest_path: str, search_dirs: List[str] = None) -> int:
    """Исправляет пути к аудиофайлам в манифесте, если они неверные."""
    if search_dirs is None:
        search_dirs = [
            OUTPUT_DIR,
            "/tf/nspeganov/sova_data",
            "/tf/nspeganov/sber_golos_data",
            "/share/audio_data/sova",
            "/share/audio_data/sber-golos",
        ]
    
    # Загружаем манифест
    manifest_data = []
    with open(manifest_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                manifest_data.append(json.loads(line.strip()))
            except:
                continue
    
    fixed_count = 0
    
    # Исправляем пути
    for item in manifest_data:
        audio_path = item.get('audio_filepath', '')
        if not audio_path or not os.path.exists(audio_path):
            # Пробуем найти файл по имени
            basename = os.path.basename(audio_path)
            if basename:
                for search_dir in search_dirs:
                    if os.path.exists(search_dir):
                        found = glob.glob(os.path.join(search_dir, "**", basename), recursive=True)
                        if found:
                            item['audio_filepath'] = os.path.abspath(found[0])
                            fixed_count += 1
                            break
    
    # Сохраняем исправленный манифест
    if fixed_count > 0:
        backup_path = manifest_path + ".backup"
        if not os.path.exists(backup_path):
            import shutil
            shutil.copy2(manifest_path, backup_path)
        
        with open(manifest_path, 'w', encoding='utf-8') as f:
            for item in manifest_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        print(f"✓ Исправлено {fixed_count} путей в манифесте")
        print(f"  Резервная копия: {backup_path}")
    
    return fixed_count

def evaluate_model(manifest_path: str, num_examples: int = 10):
    """Оценивает модель на примерах из манифеста."""
    # Сначала проверяем и исправляем пути в манифесте
    print("Проверка путей в манифесте...")
    fixed = fix_audio_paths_in_manifest(manifest_path)
    if fixed > 0:
        print(f"✓ Исправлено {fixed} путей")
    
    # Загружаем данные
    test_data = []
    with open(manifest_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                test_data.append(json.loads(line.strip()))
            except:
                continue
    
    if not test_data:
        print("⚠️  Манифест пуст или не может быть прочитан")
        return [], []
    
    # Берем случайные примеры
    sample_data = random.sample(test_data, min(num_examples, len(test_data)))
    
    wers = []
    cers = []
    
    import time
    start_time = time.time()
    
    print(f"\n{'='*60}")
    print(f"ТЕСТИРОВАНИЕ МОДЕЛИ НА {len(sample_data)} ПРИМЕРАХ")
    print(f"{'='*60}\n")
    
    # Диагностика путей перед началом
    print("📋 Диагностика путей к аудиофайлам (первые 3 примера):")
    for i, item in enumerate(sample_data[:3]):
        audio_path = item.get('audio_filepath', '')
        print(f"  Пример {i+1}:")
        print(f"    Путь в манифесте: {audio_path}")
        print(f"    Существует: {os.path.exists(audio_path) if audio_path else False}")
        if audio_path and not os.path.exists(audio_path):
            # Пробуем найти файл
            basename = os.path.basename(audio_path)
            print(f"    Имя файла: {basename}")
            # Ищем в возможных местах
            possible_dirs = [
                os.path.dirname(audio_path),
                OUTPUT_DIR,
                "/tf/nspeganov/sova_data",
                "/tf/nspeganov/sber_golos_data",
                "/share/audio_data/sova",
                "/share/audio_data/sber-golos",
            ]
            for search_dir in possible_dirs:
                if os.path.exists(search_dir):
                    found = glob.glob(os.path.join(search_dir, "**", basename), recursive=True)
                    if found:
                        print(f"    ✓ Найден в: {found[0]}")
                        break
    
    print(f"\n🚀 Начинаем распознавание...\n")
    
    # Статистика в реальном времени
    stats = {
        'processed': 0,
        'found': 0,
        'transcribed': 0,
        'errors': 0,
    }
    
    # Создаем прогресс-бар с дополнительной информацией
    pbar = tqdm(sample_data, desc="Распознавание", unit="файл")
    
    for i, item in enumerate(pbar):
        step_start = time.time()
        audio_path = item.get('audio_filepath', '')
        true_text = item.get('text', '')
        
        # Обновляем описание прогресс-бара
        pbar.set_postfix({
            'Обработано': stats['processed'],
            'Найдено': stats['found'],
            'Распознано': stats['transcribed'],
            'Ошибок': stats['errors']
        })
        
        stats['processed'] += 1
        
        if not audio_path:
            stats['errors'] += 1
            if i < 3:
                pbar.write(f"⚠️  [{i+1}/{len(sample_data)}] Отсутствует путь к аудиофайлу")
            continue
        
        # Проверяем существование файла
        if not os.path.exists(audio_path):
            # Пробуем найти файл по имени
            basename = os.path.basename(audio_path)
            if basename:
                # Ищем в возможных местах
                found_files = []
                search_dirs = [
                    OUTPUT_DIR,
                    "/tf/nspeganov/sova_data",
                    "/tf/nspeganov/sber_golos_data",
                    "/share/audio_data/sova",
                    "/share/audio_data/sber-golos",
                ]
                for search_dir in search_dirs:
                    if os.path.exists(search_dir):
                        found = glob.glob(os.path.join(search_dir, "**", basename), recursive=True)
                        if found:
                            found_files.extend(found)
                            break
                
                if found_files:
                    audio_path = found_files[0]
                    stats['found'] += 1
                    if i < 3:
                        pbar.write(f"✓ [{i+1}/{len(sample_data)}] Файл найден по имени: {os.path.basename(audio_path)}")
                else:
                    stats['errors'] += 1
                    if i < 3:
                        pbar.write(f"✗ [{i+1}/{len(sample_data)}] Файл не найден: {basename}")
                    continue
            else:
                stats['errors'] += 1
                if i < 3:
                    pbar.write(f"✗ [{i+1}/{len(sample_data)}] Неверный путь: {audio_path[:50]}...")
                continue
        else:
            stats['found'] += 1
        
        # Распознаем
        transcribe_start = time.time()
        # Включаем подробные логи только для первых 3 примеров
        verbose_log = (i < 3)
        if verbose_log:
            pbar.write(f"  🔍 Обработка файла {i+1}/{len(sample_data)}: {os.path.basename(audio_path)}")
        predicted_text = transcribe_audio(audio_path, verbose=verbose_log)
        transcribe_time = time.time() - transcribe_start
        
        # Вычисляем метрики
        if true_text and predicted_text:
            w = wer(true_text, predicted_text)
            c = cer(true_text, predicted_text)
            wers.append(w)
            cers.append(c)
            stats['transcribed'] += 1
            
            # Выводим первые несколько примеров
            if i < 5:
                pbar.write(f"\n📝 Пример {i+1}/{len(sample_data)}:")
                pbar.write(f"  Истинный текст: {true_text[:60]}...")
                pbar.write(f"  Распознанный: {predicted_text[:60]}...")
                pbar.write(f"  WER: {w:.3f}, CER: {c:.3f} (время: {transcribe_time:.2f}с)")
                
                # Воспроизведение аудио
                try:
                    display(Audio(audio_path, autoplay=False))
                except:
                    pass
        elif not predicted_text:
            stats['errors'] += 1
            if i < 3:
                pbar.write(f"⚠️  [{i+1}/{len(sample_data)}] Пустой результат распознавания")
        
        # Показываем среднее время обработки каждые 5 примеров
        if (i + 1) % 5 == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i + 1)
            remaining = avg_time * (len(sample_data) - i - 1)
            pbar.write(f"⏱️  Прогресс: {i+1}/{len(sample_data)} | "
                      f"Среднее время: {avg_time:.2f}с/файл | "
                      f"Осталось: ~{remaining:.0f}с")
    
    pbar.close()
    
    total_time = time.time() - start_time
    print(f"\n✅ Обработка завершена за {total_time:.1f} секунд")
    print(f"   Статистика: обработано {stats['processed']}, найдено {stats['found']}, "
          f"распознано {stats['transcribed']}, ошибок {stats['errors']}")
    
    # Статистика
    if wers:
        print(f"\n{'='*60}")
        print("РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ")
        print(f"{'='*60}")
        print(f"Средний WER: {np.mean(wers):.3f} ± {np.std(wers):.3f}")
        print(f"Медианный WER: {np.median(wers):.3f}")
        print(f"Средний CER: {np.mean(cers):.3f} ± {np.std(cers):.3f}")
        print(f"Медианный CER: {np.median(cers):.3f}")
        print(f"\nОбработано успешно: {len(wers)}/{len(sample_data)} примеров")
    else:
        print("\n⚠️  Не удалось вычислить метрики")
        print("\nВозможные причины:")
        print("  1. Модель не загружена или не работает")
        print("  2. Функция transcribe_audio возвращает пустые результаты")
        print("  3. Аудиофайлы не найдены или недоступны")
        print("  4. Тексты в манифесте пустые")
        
        # Диагностика
        print("\nДиагностика:")
        print(f"  - Модель загружена: {'asr_model' in globals() and asr_model is not None}")
        if 'asr_model' in globals() and asr_model is not None:
            print(f"  - Тип модели: {type(asr_model)}")
        print(f"  - Обработано примеров: {len(sample_data)}")
        found_files = sum(1 for item in sample_data if os.path.exists(item.get('audio_filepath', '')))
        print(f"  - Найдено аудиофайлов: {found_files}/{len(sample_data)}")
        
        # Показываем примеры путей
        print(f"\nПримеры путей из манифеста:")
        for i, item in enumerate(sample_data[:5]):
            audio_path = item.get('audio_filepath', '')
            exists = os.path.exists(audio_path) if audio_path else False
            status = "✓" if exists else "✗"
            print(f"  {status} {i+1}: {audio_path[:80]}...")
            if not exists and audio_path:
                basename = os.path.basename(audio_path)
                print(f"      Имя файла: {basename}")
        
        # Пробуем распознать один файл для диагностики
        if sample_data:
            test_item = sample_data[0]
            test_audio = test_item.get('audio_filepath', '')
            if test_audio:
                if os.path.exists(test_audio):
                    print(f"\nТест распознавания на первом файле: {test_audio}")
                    test_result = transcribe_audio(test_audio)
                    print(f"  Результат: '{test_result}'")
                    print(f"  Длина результата: {len(test_result)}")
                    if not test_result:
                        print("  ⚠️  Результат пустой - модель не распознает аудио")
                else:
                    print(f"\n⚠️  Первый файл не найден: {test_audio}")
                    print(f"   Попробуйте исправить пути в манифесте или проверить доступность данных")
    
    return wers, cers

# Тестируем на валидационной выборке
wers, cers = evaluate_model(val_manifest, num_examples=20)


## Сохранение модели в формате .nemo


In [ ]:
# Сохранение модели в формате .nemo
final_model_path = os.path.join(MODEL_DIR, "final_asr_model.nemo")

try:
    asr_model.save_to(final_model_path)
    print(f"✓ Модель сохранена: {final_model_path}")
    print(f"  Размер: {os.path.getsize(final_model_path) / (1024*1024):.2f} MB")
except Exception as e:
    print(f"⚠️  Ошибка при сохранении модели: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'='*60}")
print("ФАЙЛЫ ДЛЯ СДАЧИ ЗАДАНИЯ")
print(f"{'='*60}")
print(f"\n📦 Необходимо приложить:")
print(f"\n1. Модель (обязательно):")
print(f"   {final_model_path}")
print(f"\n2. Токенизатор:")
print(f"   {tokenizer_path}")
print(f"\n3. Манифесты:")
print(f"   {train_manifest}")
print(f"   {val_manifest}")
print(f"\n4. Графики обучения:")
print(f"   {os.path.join(OUTPUT_DIR, 'asr_training_metrics.png')}")


## Анализ результатов


In [ ]:
print(f"\n{'='*60}")
print("АНАЛИЗ КАЧЕСТВА МОДЕЛИ ASR")
print(f"{'='*60}")

print("\n📊 Основные результаты:")
print(f"- Размер словаря токенизатора: {actual_vocab_size} токенов")
print(f"- Целевой размер: {TARGET_VOCAB_SIZE} токенов")
if 'wers' in globals() and wers:
    print(f"- Средний WER: {np.mean(wers):.3f}")
    print(f"- Средний CER: {np.mean(cers):.3f}")

print("\n💡 Выводы:")
print("Модель ASR успешно обучена на данных из СОВА и СберГолос.")
print("Токенизатор соответствует требованиям задания (369 токенов).")
print("Модель готова к использованию для распознавания русской речи.")


In [ ]:
# Загрузка предобученной модели NeMo ASR и дообучение
print("="*60)
print("ЗАГРУЗКА МОДЕЛИ NeMo ASR")
print("="*60)

# Сначала проверяем доступные модели
print("\nПроверка доступных моделей NeMo...")
try:
    available_models = nemo_asr.models.EncDecCTCModel.list_available_models()
    print(f"✓ Найдено {len(available_models)} доступных моделей")
    
    # Ищем русские модели
    ru_models = [m for m in available_models if 'ru' in m.lower() or 'russian' in m.lower()]
    if ru_models:
        print(f"\nРусские модели ({len(ru_models)}):")
        for model in ru_models[:10]:  # Показываем первые 10
            print(f"  - {model}")
        if len(ru_models) > 10:
            print(f"  ... и еще {len(ru_models) - 10} моделей")
    else:
        print("\n⚠️  Русские модели не найдены, показываем все доступные:")
        for model in available_models[:20]:
            print(f"  - {model}")
        if len(available_models) > 20:
            print(f"  ... и еще {len(available_models) - 20} моделей")
except Exception as e:
    print(f"⚠️  Не удалось получить список моделей: {e}")

# Пробуем загрузить модель (пробуем разные варианты имен)
print("\n" + "="*60)
print("ПОПЫТКА ЗАГРУЗКИ ПРЕДОБУЧЕННОЙ МОДЕЛИ")
print("="*60)

model_names_to_try = [
    "stt_ru_conformer_ctc_large",
    "stt_ru_conformer_ctc_medium",
    "stt_ru_conformer_ctc_small",
    "stt_ru_quartznet15x5",
    "stt_ru_quartznet15x5_ctc",
    "nvidia/stt_ru_conformer_ctc_large",
    "nvidia/stt_ru_quartznet15x5",
]

asr_model = None
loaded_model_name = None

for model_name in model_names_to_try:
    try:
        print(f"\nПробуем загрузить: {model_name}")
        asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name=model_name)
        loaded_model_name = model_name
        print(f"✓ Модель успешно загружена: {model_name}")
        break
    except Exception as e:
        print(f"  ✗ Не удалось: {str(e)[:100]}...")
        continue

# Если не удалось загрузить предобученную модель
if asr_model is None:
    print("\n" + "="*60)
    print("⚠️  НЕ УДАЛОСЬ ЗАГРУЗИТЬ ПРЕДОБУЧЕННУЮ МОДЕЛЬ")
    print("="*60)
    print("\nВозможные варианты:")
    print("1. Использовать другую доступную модель из списка выше")
    print("2. Создать модель с нуля (требует больше времени на обучение)")
    print("3. Проверить подключение к интернету для загрузки моделей")
    print("\nПробуем создать базовую модель с нуля...")
    
    try:
        from omegaconf import OmegaConf
        asr_config = {
            "model": {
                "sample_rate": SAMPLE_RATE,
                "train_ds": {
                    "manifest_filepath": train_manifest,
                    "sample_rate": SAMPLE_RATE,
                    "batch_size": 16,
                    "shuffle": True,
                    "num_workers": 4,
                },
                "validation_ds": {
                    "manifest_filepath": val_manifest,
                    "sample_rate": SAMPLE_RATE,
                    "batch_size": 16,
                    "shuffle": False,
                    "num_workers": 4,
                },
            }
        }
        asr_model = nemo_asr.models.EncDecCTCModel(cfg=OmegaConf.create(asr_config["model"]))
        print("✓ Базовая модель создана")
    except Exception as e:
        print(f"✗ Не удалось создать модель: {e}")
        import traceback
        traceback.print_exc()
        raise

# Настраиваем пути к данным
if asr_model:
    asr_model._cfg.train_ds.manifest_filepath = train_manifest
    asr_model._cfg.validation_ds.manifest_filepath = val_manifest
    print("\n✓ Пути к данным настроены")
    print(f"  Train: {train_manifest}")
    print(f"  Val: {val_manifest}")
    print("\n⚠️  NeMo требует специальный формат токенизатора.")
    print("   Наш токенизатор будет использован для предобработки текста.")


In [ ]:
# Обучение модели
print("Начало обучения...")
print(f"Train манифест: {train_manifest}")
print(f"Val манифест: {val_manifest}")

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(MODEL_DIR, "checkpoints"),
    filename="asr-{epoch:02d}-{val_loss:.2f}",
    monitor="val_loss",
    mode="min",
    save_top_k=3,
    save_last=True
)

trainer = Trainer(
    max_epochs=50,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    precision=16 if torch.cuda.is_available() else 32,
    callbacks=[checkpoint_callback],
    enable_progress_bar=True,
    log_every_n_steps=50,
    default_root_dir=MODEL_DIR
)

# Запускаем обучение
try:
    trainer.fit(asr_model)
    print("\n✓ Обучение завершено!")
except Exception as e:
    print(f"\n⚠️  Ошибка при обучении: {e}")
    import traceback
    traceback.print_exc()